# Práctica de procesamiento de lenguaje natural

### Inteligencia Artificial
### Grado en Ingeniería Informática - Ingeniería del Software
### Universidad de Sevilla

Los sistemas de texto predictivo son una tecnología de entrada de texto diseñada para dispositivos móviles. Esta tecnología permite formar palabras presionando una zona de la pantalla asociada a un grupo de letras. La aplicación principal de esta tecnología es simplificar la escritura de mensajes de texto.

Aunque esta tecnología se utilizó inicialmente para facilitar la escritura de mensajes en teléfonos móviles con teclado numérico, la aparición del smartphone con teclado ampliado provocó que cambiase el tipo de aplicaciones a las que se aplica. Actualmente se utiliza tanto para facilitar la escritura en teclados ampliados (samsung swype) como para sugerir nuevas entradas de texto (escritura inteligente). También es notable su uso en dispositivos móviles más pequeños como los Smart Watch.

En la pantalla de un Smart Watch, un teclado podría consistir en agrupar las letras del alfabeto de la siguiente manera:

<img src="teclado-touchone.png"/>

Esta es una imagen del teclado de la aplicación TouchOne de 2016.

Para escribir texto con este teclado hay que pulsar las zonas asociadas a cada letra para componer las palabras. Por ejemplo, para escribir la palabra *Hola* se debería:

* Pulsar la esquina superior derecha (h).
* Pulsar el lateral derecho (o).
* Pulsar el lateral izquierdo (l).
* Pulsar la esquina superior izquierda (a).

Los sistemas de texto predicen la palabra que queremos escribir a partir de las pulsaciones realizadas. Para ello han efectuado previamente un análisis estadístico de un corpus de textos de referencia, determinando las probabilidades de las correspondencias entre distintas secuencias de pulsaciones y posibles palabras.

Estos sistemas suelen mostrar la palabra que corresponde con mayor probabilidad a la combinación de pulsaciones realizada por el usuario, actualizándose esta palabra a medida que el usuario realiza nuevas pulsaciones. Además, es habitual ofrecer al usuario la posibilidad de requerir otras posibilidades, aparte de aceptar la palabra propuesta por el sistema.

La efectividad de un sistema de prediccion de texto dependerá de varios factores:

* La calidad del corpus.
* El modelo de lenguaje obtenido a partir del análisis estadístico del corpus.

En esta práctica se propone la implementación de un sistema de predicción de texto para un teclado similar al de la aplicación TouchOne mostrada antes. Cada bloque de letras tendrá asociado un número del 1 al 8 de la siguiente forma:
1. a, á, b, c
2. d, e, é, f
3. g, h, i, í
4. j, k, l
5. m, n, ñ, o, ó
6. p, q, r, s
7. t, u, ú, ü, v
8. w, x, y, z

De esta forma, si por ejemplo se pretendiera escribir la frase `algoritmo de texto predictivo`, la entrada al sistema sería la secuencia de números (y espacios en blanco) `143563755 22 72875 6622317375`.

In [1]:
bloques_de_letras = {'1': 'aábc', '2':'deéf', '3': 'ghií', '4': 'jkl',
                     '5': 'mnñoó', '6': 'pqrs', '7': 'tuúüv', '8': 'wxyz'}

In [2]:
def codifica_palabra(palabra):
    códigos = []
    for letra in palabra:
        for código, bloque in bloques_de_letras.items():
            if letra in bloque:
                códigos.append(código)
                break
    return ''.join(códigos)

In [3]:
codifica_palabra('algoritmo')

'143563755'

## La plataforma NLTK

Para la realización de esta práctica nos apoyaremos en [NLTK](https://www.nltk.org), que es una plataforma de Python que proporciona un conjunto de bibliotecas para el procesamiento del lenguaje natural, junto con interfaces fáciles de usar a multitud de corpus y recursos léxicos.

A continuación vamos a ilustrar con un ejemplo sencillo el uso de NLTK para la construcción de [modelos de lenguajes basados en n-gramas](https://www.nltk.org/api/nltk.lm.html).

En primer lugar establecemos cuál será nuestro vocabulario de términos, en este caso formado por las letras a, b y c y los símbolos de inicio y fin de frase.

In [4]:
from nltk.lm.vocabulary import Vocabulary

vocabulario = Vocabulary(['a', 'b', 'c', '<s>', '</s>'])

Para representar todos los términos desconocidos que nos podamos encontrar en un texto se añade automáticamente el término `'<UNK>'` (de *unknown*, desconocido en inglés).

In [5]:
'<UNK>' in vocabulario

True

Para buscar un término en el vocabulario basta usar el método `lookup`.

In [6]:
print(vocabulario.lookup('a'))
print(vocabulario.lookup('e'))

a
<UNK>


Ahora vamos a establecer un corpus de entrenamiento y un corpus de prueba. En NLTK las «frases» deben ser listas de términos y los corpus listas de frases.

In [7]:
corpus_entrenamiento = [['a', 'b', 'b', 'b'], ['d', 'a', 'b'], ['a', 'b', 'a', 'e']]
corpus_prueba = [['a', 'b', 'a', 'b'], ['a', 'b']]

Para determinar los posibles n-gramas de una frase se puede utilizar la función `ngrams`. Existen también las funciones `bigrams` y `trigrams` que, como indican sus nombres, permiten determinar los bigramas y trigramas de una frase. Finalmente, la función `everygrams` determina todos los posibles n-gramas hasta el orden especificado.

*Nota*: estas funciones devuelven [iteradores](https://docs.python.org/es/3/tutorial/classes.html#iterators), que básicamente son secuencias *perezosas*, en las que sus elementos no se determinan hasta que son requeridos, evitando de esta forma tener que almacenarlos a todos en memoria. Es por ello que para poder observar los resultados de los ejemplos siguientes se construyen listas con los elementos de los iteradores obtenidos.

In [8]:
from nltk.util import ngrams, bigrams, trigrams, everygrams

print(list(ngrams(['a', 'b', 'a', 'e'], n=1)))
print(list(bigrams(['a', 'b', 'a', 'e'])))
print(list(trigrams(['a', 'b', 'a', 'e'])))
print(list(everygrams(['a', 'b', 'a', 'e'], min_len=2, max_len=3)))

[('a',), ('b',), ('a',), ('e',)]
[('a', 'b'), ('b', 'a'), ('a', 'e')]
[('a', 'b', 'a'), ('b', 'a', 'e')]
[('a', 'b'), ('a', 'b', 'a'), ('b', 'a'), ('b', 'a', 'e'), ('a', 'e')]


En el caso de los n-gramas de orden mayor que 1, es conveniente delimitar las frases con los símbolos de inicio y fin de frase.

In [9]:
from nltk.util import pad_sequence

print(list(pad_sequence(['a', 'b', 'a', 'e'], n=3,  # la secuencia se delimita con n-1 símbolos
                        pad_left=True, left_pad_symbol='<s>',
                        pad_right=True, right_pad_symbol='</s>')))
print(list(everygrams(['a', 'b', 'a', 'e'], min_len=2, max_len=3,
                      pad_left=True, left_pad_symbol='<s>',
                      pad_right=True, right_pad_symbol='</s>')))

['<s>', '<s>', 'a', 'b', 'a', 'e', '</s>', '</s>']
[('<s>', '<s>'), ('<s>', '<s>', 'a'), ('<s>', 'a'), ('<s>', 'a', 'b'), ('a', 'b'), ('a', 'b', 'a'), ('b', 'a'), ('b', 'a', 'e'), ('a', 'e'), ('a', 'e', '</s>'), ('e', '</s>'), ('e', '</s>', '</s>'), ('</s>', '</s>')]


Para estimar, mediante máxima verosimilitud, la probabilidad de los distintos n-gramas, basta instanciar la clase `MLE`. Hay que proporcionar el máximo orden de los n-gramas a considerar y, opcionalmente, el vocabulario de términos. En caso de que este último no se proporcione, también se podrá inferir a partir de los términos que aparezcan en un corpus de texto proporcionado al entrenar, mediante el método `fit`, el modelo.

In [10]:
from nltk.lm import MLE

modelo_lenguaje = MLE(3, vocabulary=vocabulario)
ngramas = (everygrams(frase, min_len=2, max_len=3,
                      pad_left=True, left_pad_symbol='<s>',
                      pad_right=True, right_pad_symbol='</s>')
           for frase in corpus_entrenamiento)
modelo_lenguaje.fit(ngramas)

Las probabilidades estimadas de los unigramas son todas iguales a 0, ya que no hemos proporcionado estos.

In [11]:
{t: modelo_lenguaje.score(t) for t in vocabulario}

{'a': 0, 'b': 0, 'c': 0, '<s>': 0, '</s>': 0, '<UNK>': 0}

No ocurre lo mismo para los bigramas.

In [12]:
{('a', t): modelo_lenguaje.score(t, context=['a']) for t in vocabulario}

{('a', 'a'): 0.0,
 ('a', 'b'): 0.75,
 ('a', 'c'): 0.0,
 ('a', '<s>'): 0.0,
 ('a', '</s>'): 0.0,
 ('a', '<UNK>'): 0.25}

Tampoco para los trigramas. En este caso mostramos el logaritmo de la probabilidad estimada, que es lo que se debe usar en productos de probabilidades para evitar desbordamientos numéricos.

In [13]:
{('a', 'b', t): modelo_lenguaje.logscore(t, context=['a', 'b']) for t in vocabulario}

{('a', 'b', 'a'): -1.5849625007211563,
 ('a', 'b', 'b'): -1.5849625007211563,
 ('a', 'b', 'c'): -inf,
 ('a', 'b', '<s>'): -inf,
 ('a', 'b', '</s>'): -1.5849625007211563,
 ('a', 'b', '<UNK>'): -inf}

Finalmente, calculamos la perplejidad del modelo sobre el corpus de prueba. Para ello debemos proporcionar una secuencia plana de los n-gramas a evaluar, para lo que es útil la función `flatten`.

In [14]:
from nltk.lm.preprocessing import flatten

ngramas = (bigrams(frase,
                   pad_left=True, left_pad_symbol='<s>',
                   pad_right=True, right_pad_symbol='</s>')
           for frase in corpus_prueba)
modelo_lenguaje.perplexity(flatten(ngramas))

2.254180002028708

## Predicción de texto a partir de modelos de $n$-gramas de letras

En esta primera parte de la práctica se pretende construir un sistema de predicción de texto basado en un modelo de lenguaje construido a partir de modelos de $n$-gramas de letras.

En primer lugar se debe cargar un corpus de textos en español que nos permita entrenar el modelo. Se puede, por ejemplo, utilizar el corpus proporcionado en el fichero `corpus.txt`.

NLTK proporciona diferentes clases que permiten leer corpus de muy diversos tipos. Entre ellas se encuentra la clase `PlaintextCorpusReader` para leer corpus proporcionados en ficheros de texto plano, como es el caso que nos ocupa. Esta clase identificará las frases y palabras contenidas en el corpus, en un proceso conocido como *tokenización* (es decir, identificación de los *tokens* que componen el texto).

NLTK es suficientemente flexible como para permitirnos elegir los algoritmos que separan el texto en párrafos, frases y palabras, pero nosotros usaremos los algoritmos por defecto: las líneas en blanco separan los párrafos; las frases se separan en palabras formadas bien por secuencias de caracteres alfanuméricos, bien por secuencias de caracteres no alfanuméricos ni espacios (que se descartan); los párrafos se separan en frases usando un modelo entrenado mediante aprendizaje no supervisado que se ha comprobado que funciona bien para muchos lenguajes europeos.

Para poder usar este último modelo es necesario descargarlo primero.

In [15]:
from nltk import download

download('punkt', download_dir='.')

[nltk_data] Downloading package punkt to ....
[nltk_data]   Package punkt is already up-to-date!


True

Ahora ya podemos leer el corpus de entrenamiento (que, aunque no es el caso, podría estar dividido en varios ficheros de texto), estableciendo como identificador de frases el modelo entrenado para el español. Establecemos también explícitamente [UTF-8](https://es.wikipedia.org/wiki/UTF-8) como codificación de caracteres para evitar problemas en sistemas Windows.

In [16]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from nltk.data import load

corpus_entrenamiento = PlaintextCorpusReader(root='.', fileids=['corpus.txt'], encoding='utf8',
                                             sent_tokenizer=load('tokenizers/punkt/spanish.pickle'))

El método `sents` proporciona las frases identificadas en el corpus (como devuelve un iterador, hacemos uso de la biblioteca estándar *itertools* para manejarlo con comodidad).

In [17]:
import itertools

for frase in itertools.islice(corpus_entrenamiento.sents(), 5):
    print(frase)

['¿', 'Quién', 'o', 'qué', 'mutila', 'y', 'mata', 'a', 'los', 'niños', 'de', 'un', 'pequeño', 'pueblo', 'norteamericano', '?']
['¿', 'Por', 'qué', 'llega', 'cíclicamente', 'el', 'horror', 'a', 'Derry', 'en', 'forma', 'de', 'un', 'payaso', 'siniestro', 'que', 'va', 'sembrando', 'la', 'destrucción', 'a', 'su', 'paso', '?']
['Esto', 'es', 'lo', 'que', 'se', 'proponen', 'averiguar', 'los', 'protagonistas', 'de', 'esta', 'novela', '.']
['Tras', 'veintisiete', 'años', 'de', 'tranquilidad', 'y', 'lejanía', 'una', 'antigua', 'promesa', 'infantil', 'les', 'hace', 'volver', 'al', 'lugar', 'en', 'el', 'que', 'vivieron', 'su', 'infancia', 'y', 'juventud', 'como', 'una', 'terrible', 'pesadilla', '.']
['Regresan', 'a', 'Derry', 'para', 'enfrentarse', 'con', 'su', 'pasado', 'y', 'enterrar', 'definitivamente', 'la', 'amenaza', 'que', 'los', 'amargó', 'durante', 'su', 'niñez', '.']


De forma análoga, el método `words` proporciona las palabras identificadas en el corpus.

In [18]:
for palabra in itertools.islice(corpus_entrenamiento.words(), 10):
    print(palabra)

¿
Quién
o
qué
mutila
y
mata
a
los
niños


Para separar una palabra en la lista de sus caracteres basta usar la función `list` estándar de Python.

In [19]:
list('norteamericano')

['n', 'o', 'r', 't', 'e', 'a', 'm', 'e', 'r', 'i', 'c', 'a', 'n', 'o']

En base a la descripción del sistema de predicción de texto que se quiere construir, el vocabulario estará formado por las letras del alfabeto español, junto con los símbolos de inicio y fin de palabra (en esta parte de la práctica las secuencias que manejamos son las distintas palabras del corpus) y el símbolo de letra desconocida.

In [20]:
letras = [letra for bloque in bloques_de_letras.values() for letra in bloque]
inicio_palabra = '<s>'
fin_palabra = '</s>'
vocabulario_letras = Vocabulary(letras + [inicio_palabra, fin_palabra])
print(list(vocabulario_letras))

['a', 'á', 'b', 'c', 'd', 'e', 'é', 'f', 'g', 'h', 'i', 'í', 'j', 'k', 'l', 'm', 'n', 'ñ', 'o', 'ó', 'p', 'q', 'r', 's', 't', 'u', 'ú', 'ü', 'v', 'w', 'x', 'y', 'z', '<s>', '</s>', '<UNK>']


#### **Ejercicio 1**: construir un modelo de unigramas de letras entrenado con el corpus proporcionado.

Nota: para este modelo no es necesario delimitar las palabras con los símbolos especiales de inicio y fin ya que únicamente introducirán un factor multiplicativo al estimar la probabilidad de las distintas palabras, por lo que no afectará a la comparación entre esas probabilidades que, como veremos, es lo que se buscará realizar.

In [21]:
modelo_unigrama_letras = MLE(1, vocabulary=vocabulario_letras)
unigramas_letras = (ngrams(list(palabra), n=1)
                    for palabra in corpus_entrenamiento.words())
modelo_unigrama_letras.fit(unigramas_letras)

Una vez construido el modelo de lenguaje, podemos seguir tres estrategias a la hora de predecir qué palabra ha tratado de introducir el usuario a partir de la secuencia de pulsaciones realizada.

#### **Ejercicio 2**: definir una función que reciba una cadena de pulsaciones de teclas y devuelva una cadena construida con las letras más probables correspondientes a esas pulsaciones.

In [22]:
def letras_más_probables(pulsaciones):
    letras = []
    for pulsación in pulsaciones:
        bloque = bloques_de_letras[pulsación]
        letra_más_probable = max(bloque, key=modelo_unigrama_letras.score)
        letras.append(letra_más_probable)
    return ''.join(letras)

In [23]:
letras_más_probables(codifica_palabra('grito'))

'isito'

#### **Ejercicio 3**: definir una función que reciba una cadena de pulsaciones de teclas y devuelva la cadena más probable compatible con esas pulsaciones.

Ayuda: considérese el uso de la función `itertools.product` para determinar todas las cadenas compatibles.

In [24]:
def palabra_más_probable(pulsaciones):
    palabra_más_probable = ''
    mayor_log_probabilidad = float('-inf')
    bloques = (bloques_de_letras[pulsación] for pulsación in pulsaciones)
    for palabra in itertools.product(*bloques):
        log_probabilidad_palabra = sum(modelo_unigrama_letras.logscore(letra)
                                       for letra in palabra)
        if log_probabilidad_palabra > mayor_log_probabilidad:
            palabra_más_probable = palabra
            mayor_log_probabilidad = log_probabilidad_palabra
    return ''.join(palabra_más_probable)

In [25]:
palabra_más_probable(codifica_palabra('grito'))

'isito'

#### **Ejercicio 4**: definir una función que reciba una cadena de pulsaciones de teclas y una cantidad $n$ de palabras y devuelva las $n$ cadenas más probables compatibles con esas pulsaciones, en orden decreciente de probabilidad.

In [26]:
def palabras_más_probables(pulsaciones, n):
    palabras = {}
    bloques = (bloques_de_letras[pulsación] for pulsación in pulsaciones)
    for palabra in itertools.product(*bloques):
        log_probabilidad_palabra = sum(modelo_unigrama_letras.logscore(letra)
                                       for letra in palabra)
        palabras[''.join(palabra)] = log_probabilidad_palabra
    return sorted(palabras, key=palabras.get, reverse=True)[:n]

In [27]:
palabras_más_probables(codifica_palabra('grito'), 5)

['isito', 'irito', 'isiuo', 'iriuo', 'isitn']

El modelo unigrama de letras no proporciona resultados satisfactorios, ya que no utiliza la información del contexto.

#### **Ejercicio 5**: repite los ejercicios 1 a 4 anteriores, pero construyendo modelos bigramas y trigramas de letras, en lugar de un modelo unigrama.

In [28]:
modelo_bigrama_letras = MLE(2, vocabulary=vocabulario_letras)
bigramas_letras = (bigrams(list(palabra),
                           pad_left=True, left_pad_symbol='<s>',
                           pad_right=True, right_pad_symbol='</s>')
                   for palabra in corpus_entrenamiento.words())
modelo_bigrama_letras.fit(bigramas_letras)

def letras_más_probables(pulsaciones):
    letras = []
    letra_anterior = '<s>'
    for pulsación in pulsaciones:
        bloque = bloques_de_letras[pulsación]
        letra_más_probable = max(bloque,
                                 key=lambda letra: modelo_bigrama_letras.score(letra, context=[letra_anterior]))
        letras.append(letra_más_probable)
        letra_anterior = letra_más_probable
    return ''.join(letras)

print(letras_más_probables(codifica_palabra('grito')))

def palabra_más_probable(pulsaciones):
    palabra_más_probable = ''
    mayor_log_probabilidad = float('-inf')
    bloques = (bloques_de_letras[pulsación] for pulsación in pulsaciones)
    for palabra in itertools.product(*bloques):
        bigramas_palabra = bigrams(palabra,
                                   pad_left=True, left_pad_symbol='<s>',
                                   pad_right=True, right_pad_symbol='</s>')
        log_probabilidad_palabra = sum(modelo_bigrama_letras.logscore(bigrama[-1], context=bigrama[:-1])
                                       for bigrama in bigramas_palabra)
        if log_probabilidad_palabra > mayor_log_probabilidad:
            palabra_más_probable = palabra
            mayor_log_probabilidad = log_probabilidad_palabra
    return ''.join(palabra_más_probable)

print(palabra_más_probable(codifica_palabra('grito')))

def palabras_más_probables(pulsaciones, n):
    palabras = {}
    bloques = (bloques_de_letras[pulsación] for pulsación in pulsaciones)
    for palabra in itertools.product(*bloques):
        bigramas_palabra = bigrams(palabra,
                                   pad_left=True, left_pad_symbol='<s>',
                                   pad_right=True, right_pad_symbol='</s>')
        log_probabilidad_palabra = sum(modelo_bigrama_letras.logscore(bigrama[-1], context=bigrama[:-1])
                                       for bigrama in bigramas_palabra)
        palabras[''.join(palabra)] = log_probabilidad_palabra
    return sorted(palabras, key=palabras.get, reverse=True)[:n]

print(palabras_más_probables(codifica_palabra('grito'), 5))

hrito
grito
['grito', 'grgun', 'irito', 'isito', 'grivo']


In [ ]:
modelo_trigrama_letras = MLE(3, vocabulary=vocabulario_letras)
trigramas_letras = (trigrams(list(palabra),
                             pad_left=True, left_pad_symbol='<s>',
                             pad_right=True, right_pad_symbol='</s>')
                    for palabra in corpus_entrenamiento.words())
modelo_trigrama_letras.fit(trigramas_letras)

def letras_más_probables(pulsaciones):
    letras = []
    letras_anteriores = ['<s>', '<s>']
    for pulsación in pulsaciones:
        bloque = bloques_de_letras[pulsación]
        letra_más_probable = max(bloque,
                                 key=lambda letra: modelo_trigrama_letras.score(letra, context=letras_anteriores))
        letras.append(letra_más_probable)
        letras_anteriores.append(letra_más_probable)
        letras_anteriores.pop(0)
    return ''.join(letras)

print(letras_más_probables(codifica_palabra('grito')))

def palabra_más_probable(pulsaciones):
    palabra_más_probable = ''
    mayor_log_probabilidad = float('-inf')
    bloques = (bloques_de_letras[pulsación] for pulsación in pulsaciones)
    for palabra in itertools.product(*bloques):
        trigramas_palabra = trigrams(palabra,
                                     pad_left=True, left_pad_symbol='<s>',
                                     pad_right=True, right_pad_symbol='</s>')
        log_probabilidad_palabra = sum(modelo_trigrama_letras.logscore(trigrama[-1], context=trigrama[:-1])
                                       for trigrama in trigramas_palabra)
        if log_probabilidad_palabra > mayor_log_probabilidad:
            palabra_más_probable = palabra
            mayor_log_probabilidad = log_probabilidad_palabra
    return ''.join(palabra_más_probable)

print(palabra_más_probable(codifica_palabra('grito')))

def palabras_más_probables(pulsaciones, n):
    palabras = {}
    bloques = (bloques_de_letras[pulsación] for pulsación in pulsaciones)
    for palabra in itertools.product(*bloques):
        trigramas_palabra = trigrams(palabra,
                                     pad_left=True, left_pad_symbol='<s>',
                                     pad_right=True, right_pad_symbol='</s>')
        log_probabilidad_palabra = sum(modelo_trigrama_letras.logscore(trigrama[-1], context=trigrama[:-1])
                                       for trigrama in trigramas_palabra)
        palabras[''.join(palabra)] = log_probabilidad_palabra
    return sorted(palabras, key=palabras.get, reverse=True)[:n]

print(palabras_más_probables(codifica_palabra('grito'), 5))

## Predicción de texto a partir de modelos de $n$-gramas de palabras

Los modelos de $n$-gramas de letras no parecen proporcionar resultados satisfactorios. En esta segunda parte de la práctica se pretende construir un sistema de predicción de texto basado en un modelo de lenguaje construido a partir de modelos de $n$-gramas de palabras.

En este caso el vocabulario estará formado por todas las palabras identificadas en el corpus. Es posible establecer una cantidad mínima de ocurrencias para que una palabra se incluya o no en el vocabulario. Los símbolos de inicio y fin de secuencia delimitarán, en su caso, secuencias de palabras.

In [ ]:
inicio_frase = '<s>'
fin_frase = '</s>'
# Cada palabra del vocabulario debe ocurrir al menos 5 veces en el corpus
vocabulario_palabras = Vocabulary(corpus_entrenamiento.words(), unk_cutoff=5)
vocabulario_palabras.update([inicio_frase, fin_frase])

#### **Ejercicio 6**: construir un modelo de unigramas de palabras entrenado con el corpus proporcionado.

In [ ]:
modelo_unigrama_palabras = MLE(1, vocabulary=vocabulario_palabras)
unigramas_palabras = (ngrams(frase, n=1)
                      for frase in corpus_entrenamiento.sents())
modelo_unigrama_palabras.fit(unigramas_palabras)

#### **Ejercicio 7**: definir una función que reciba una cadena de pulsaciones de teclas y una lista de palabras anteriores como contexto (en el caso del modelo unigrama el contexto será la lista vacía) y devuelva la cadena más probable compatible con esas pulsaciones.

In [ ]:
def palabra_más_probable(pulsaciones, contexto):
    palabra_más_probable = ''
    mayor_probabilidad = float('-inf')
    bloques = (bloques_de_letras[pulsación] for pulsación in pulsaciones)
    for palabra in itertools.product(*bloques):
        palabra = ''.join(palabra)
        if palabra in vocabulario_palabras:
            probabilidad_palabra = modelo_unigrama_palabras.score(palabra)
            if probabilidad_palabra > mayor_probabilidad:
                palabra_más_probable = palabra
                mayor_probabilidad = probabilidad_palabra
    return palabra_más_probable

In [ ]:
palabra_más_probable(codifica_palabra('grito'), contexto=[])

#### **Ejercicio 8**: definir una función que reciba una cadena de pulsaciones de teclas, una lista de palabras anteriores como contexto y una cantidad $n$ de palabras y devuelva las $n$ cadenas más probables compatibles con esas pulsaciones, en orden decreciente de probabilidad.

In [ ]:
def palabras_más_probables(pulsaciones, contexto, n):
    palabras = {}
    bloques = (bloques_de_letras[pulsación] for pulsación in pulsaciones)
    for palabra in itertools.product(*bloques):
        palabra = ''.join(palabra)
        if palabra in vocabulario_palabras:
            probabilidad_palabra = modelo_unigrama_palabras.score(palabra)
            palabras[palabra] = probabilidad_palabra
    return sorted(palabras, key=palabras.get, reverse=True)[:n]

In [ ]:
palabras_más_probables(codifica_palabra('grito'), contexto=[], n=5)

#### **Ejercicio 9**: repite los ejercicios 6 a 8 anteriores, pero construyendo modelos bigramas y trigramas de palabras, en lugar de un modelo unigrama.

In [ ]:
modelo_bigrama_palabras = MLE(2, vocabulary=vocabulario_palabras)
bigramas_palabras = (bigrams(frase,
                             pad_left=True, left_pad_symbol='<s>',
                             pad_right=True, right_pad_symbol='</s>')
                    for frase in corpus_entrenamiento.sents())
modelo_bigrama_palabras.fit(bigramas_palabras)

def palabra_más_probable(pulsaciones, contexto):
    palabra_más_probable = ''
    mayor_probabilidad = float('-inf')
    bloques = (bloques_de_letras[pulsación] for pulsación in pulsaciones)
    for palabra in itertools.product(*bloques):
        palabra = ''.join(palabra)
        if palabra in vocabulario_palabras:
            probabilidad_palabra = modelo_bigrama_palabras.score(palabra, context=contexto)
            if probabilidad_palabra > mayor_probabilidad:
                palabra_más_probable = palabra
                mayor_probabilidad = probabilidad_palabra
    return palabra_más_probable

print(palabra_más_probable(codifica_palabra('grito'), contexto=['un']))

def palabras_más_probables(pulsaciones, contexto, n):
    palabras = {}
    bloques = (bloques_de_letras[pulsación] for pulsación in pulsaciones)
    for palabra in itertools.product(*bloques):
        palabra = ''.join(palabra)
        if palabra in vocabulario_palabras:
            probabilidad_palabra = modelo_bigrama_palabras.score(palabra, context=contexto)
            palabras[palabra] = probabilidad_palabra
    return sorted(palabras, key=palabras.get, reverse=True)[:n]

print(palabras_más_probables(codifica_palabra('grito'), contexto=['un'], n=5))

In [ ]:
modelo_trigrama_palabras = MLE(3, vocabulary=vocabulario_palabras)
trigramas_palabras = (trigrams(frase,
                               pad_left=True, left_pad_symbol='<s>',
                               pad_right=True, right_pad_symbol='</s>')
                      for frase in corpus_entrenamiento.sents())
modelo_trigrama_palabras.fit(trigramas_palabras)

def palabra_más_probable(pulsaciones, contexto):
    palabra_más_probable = ''
    mayor_probabilidad = float('-inf')
    bloques = (bloques_de_letras[pulsación] for pulsación in pulsaciones)
    for palabra in itertools.product(*bloques):
        palabra = ''.join(palabra)
        if palabra in vocabulario_palabras:
            probabilidad_palabra = modelo_trigrama_palabras.score(palabra, context=contexto)
            if probabilidad_palabra > mayor_probabilidad:
                palabra_más_probable = palabra
                mayor_probabilidad = probabilidad_palabra
    return palabra_más_probable

print(palabra_más_probable(codifica_palabra('grito'), contexto=['un']))

def palabras_más_probables(pulsaciones, contexto, n):
    palabras = {}
    bloques = (bloques_de_letras[pulsación] for pulsación in pulsaciones)
    for palabra in itertools.product(*bloques):
        palabra = ''.join(palabra)
        if palabra in vocabulario_palabras:
            probabilidad_palabra = modelo_trigrama_palabras.score(palabra, context=contexto)
            palabras[palabra] = probabilidad_palabra
    return sorted(palabras, key=palabras.get, reverse=True)[:n]

print(palabras_más_probables(codifica_palabra('grito'), contexto=['un'], n=5))